In [5]:
import torch
import torch.nn as nn
from kasim_model import Kasim_Model
from kasim_tokenizer import Tokenizer   

k_tokenizer=Tokenizer("tokenizer.json")
sentence = "the capital of united states and the capital of france"
tokens = k_tokenizer.encode(sentence)
tokens = torch.tensor(tokens).unsqueeze(0)  # Add batch dimension


k_model = Kasim_Model(vocab_size=len(k_tokenizer.vocab), embed_dim=4, context_length=32)
k_model


C:\Users\kasim\AppData\Local\Temp\ipykernel_25388\1443349770.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens = torch.tensor(tokens).unsqueeze(0)  # Add batch dimension


Kasim_Model(
  (embedding): Embedding(64, 4)
  (pos_embed): Embedding(32, 4)
  (self_attention): KasimCausalAttention(
    (q_weight): Linear(in_features=4, out_features=4, bias=False)
    (k_weight): Linear(in_features=4, out_features=4, bias=False)
    (v_weight): Linear(in_features=4, out_features=4, bias=False)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [6]:
sentence_meanings_whith_attention = k_model(tokens)
sentence_meanings_whith_attention

tensor([[[ 6.5979e-01, -5.7790e-01,  1.6746e-01, -6.6660e-01],
         [ 3.4240e-01, -2.7027e-01,  9.1047e-02, -4.8475e-01],
         [ 5.7518e-02, -3.1539e-02, -4.7680e-02,  1.5087e-02],
         [ 1.6371e-01, -1.3049e-01, -1.6156e-02, -7.7814e-02],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 2.2928e-01, -4.0661e-01,  4.0655e-02, -4.7999e-02],
         [ 9.8098e-02, -7.7434e-02,  2.6085e-02, -1.3888e-01],
         [ 4.0027e-02, -4.1049e-02, -2.0742e-02, -7.4604e-03],
         [ 8.4715e-02, -1.1791e-01,  7.1326e-03, -3.9499e-02],
         [ 2.4668e-01, -1.3704e-01, -1.2420e-02, -2.9597e-01],
         [-3.3944e-02,  1.3298e-01, -9.6689e-02,  4.4016e-02],
         [ 7.4738e-02,  1.7528e-01, -1.0887e-01, -1.3570e-01],
         [ 3.3311e-02,  2.5878e-01, -2.3116e-01,  1.1625e-02],
         [ 1.6578e-01, -3.4468e-02, -1.0083e-01, -1.1692e-01],
         [ 1.5113e-01, -1.7906e-01, -3.0293e-02, -5.1137e-02],
         [ 8.0528e-02, -6.2598e-02, -3.7555e-03, -7.790

In [7]:
sentence_meanings_whith_attention

tensor([[[ 6.5979e-01, -5.7790e-01,  1.6746e-01, -6.6660e-01],
         [ 3.4240e-01, -2.7027e-01,  9.1047e-02, -4.8475e-01],
         [ 5.7518e-02, -3.1539e-02, -4.7680e-02,  1.5087e-02],
         [ 1.6371e-01, -1.3049e-01, -1.6156e-02, -7.7814e-02],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 2.2928e-01, -4.0661e-01,  4.0655e-02, -4.7999e-02],
         [ 9.8098e-02, -7.7434e-02,  2.6085e-02, -1.3888e-01],
         [ 4.0027e-02, -4.1049e-02, -2.0742e-02, -7.4604e-03],
         [ 8.4715e-02, -1.1791e-01,  7.1326e-03, -3.9499e-02],
         [ 2.4668e-01, -1.3704e-01, -1.2420e-02, -2.9597e-01],
         [-3.3944e-02,  1.3298e-01, -9.6689e-02,  4.4016e-02],
         [ 7.4738e-02,  1.7528e-01, -1.0887e-01, -1.3570e-01],
         [ 3.3311e-02,  2.5878e-01, -2.3116e-01,  1.1625e-02],
         [ 1.6578e-01, -3.4468e-02, -1.0083e-01, -1.1692e-01],
         [ 1.5113e-01, -1.7906e-01, -3.0293e-02, -5.1137e-02],
         [ 8.0528e-02, -6.2598e-02, -3.7555e-03, -7.790